In [1]:
from datetime import datetime
inicio_tempo = datetime.now()

# OBJETIVO

-- EXTRAIR AS BASES DE ACOMPANHAMENTO DA FREQUENCIA POR HOSPITAL DIARIAMENTE DO QVIS, AUTOMÁTICO COM SELENIUM

-- MOVER AS BASES PARA A PASTA CORRETA DO ACOMP FREQ SOLUCX

# IMPORTS

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import os
# from datetime import datetime
import shutil
from send2trash import send2trash
import math
import pandas as pd

# LIMPAR ARQUIVOS QUE ESTEJAM NOMEADOS COMO DATA NOS DOWNLOADS

In [3]:
# Define o caminho da pasta Downloads
# downloads_folder = r"caminho downloads Gaby"
downloads_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"

# Itera sobre os arquivos na pasta
for file in os.listdir(downloads_folder):
    file_path = os.path.join(downloads_folder, file)
    
    # Verifica se o arquivo é um .xlsx e começa com "data"
    if file.startswith("data") and file.endswith(".xlsx") and os.path.isfile(file_path):
        send2trash(file_path)  # Move para a Lixeira
        print(f"Moved to Trash: {file_path}")

print("Cleanup complete.")

Cleanup complete.


# EXTRAÇÃO DAS BASES DE ACOMP FREQ POR HOSPITAL NO BI ACOMPANHAMENTO DA FREQUENCIA QVIS

In [4]:
url = "https://app.powerbi.com/groups/me/reports/d86aa107-fd90-4155-9500-7d4ee4d6e138/ReportSection69f54088ab14b0613445?experience=power-bi"

def reset_filters(driver):
    try:
        reset_icon = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "mat-icon[fonticon='pbi-glyph-reset']"))
        )
        reset_icon.click()
        print("🔹 Resetando filtros...")

        redefinir_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.ID, "okButton"))
        )
        redefinir_button.click()
        print("🔹 Filtros resetados.")

        # Aguarda o carregamento do relatório após reset
        WebDriverWait(driver, 120).until_not(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".spinner, .loading-indicator"))
        )
        time.sleep(5)  # pequeno extra para estabilizar

    except Exception as e:
        print(f"⚠️ Erro ao resetar filtros: {e}")
        return False
    return True


def safe_click(element, driver, pause=5):
    """Tenta clicar via Selenium, se falhar usa JS"""
    try:
        element.click()
    except:
        driver.execute_script("arguments[0].click();", element)
    time.sleep(pause)  # pequena pausa para estabilizar

def export_data(driver):
    try:
        # -------------------------
        # Clicar na coluna "Última Admissão"
        # -------------------------
        admissao_xpath = "//*[@id='pvExplorationHost']/div/div/exploration/div/explore-canvas/div/div[2]/div/div[2]/div[2]/visual-container-repeat/visual-container[24]/transform/div/div[3]/div/div/visual-modern/div/div/div[2]/div[1]/div[1]/div/div/div/div[10]"
        admissao_column_header = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, admissao_xpath))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", admissao_column_header)
        time.sleep(0.5)
        safe_click(admissao_column_header, driver)
        print("Clicando na coluna Última Admissão (1º clique) e esperando 60s...")
        time.sleep(60)

        # Segundo clique na mesma coluna
        admissao_column_header = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, admissao_xpath))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", admissao_column_header)
        time.sleep(0.5)
        safe_click(admissao_column_header, driver)
        print("Clicando na coluna Última Admissão (2º clique) e esperando 60s...")
        time.sleep(60)

        # -------------------------
        # Clique adicional (se necessário)
        # -------------------------
        extra_click_xpath = "//*[@id='pvExplorationHost']/div/div/exploration/div/explore-canvas/div/div[2]/div/div[2]/div[2]/visual-container-repeat/visual-container[24]/transform/div/div[3]/div/div/visual-modern/div/div/div[2]/div[1]/div[2]"
        extra_click_element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, extra_click_xpath))
        )
        safe_click(extra_click_element, driver)
        print("Clique adicional realizado. Aguardando 30s...")
        time.sleep(30)

        # -------------------------
        # Abrir menu "Mais opções"
        # -------------------------
        mais_opcoes_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='visual-more-options-btn']"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", mais_opcoes_button)
        time.sleep(5)
        safe_click(mais_opcoes_button, driver)
        print("Menu 'Mais opções' aberto. Aguardando 1s para renderização...")
        time.sleep(10)  # aguarda animação do menu

        # -------------------------
        # Clicar em "Exportar dados"
        # -------------------------
        try:
            exportar_dados_button = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, "//button[@aria-label='Exportar dados']"))
            )
            safe_click(exportar_dados_button, driver, pause=5)
            print("Botão 'Exportar dados' clicado. Aguardando 30s...")
            time.sleep(30)

        except:
            # -------------------------
            # Clicar em "Exportar dados"
            # -------------------------
            exportar_dados_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='pbimenu-item.Exportar dados']"))
            )
            safe_click(exportar_dados_button, driver)
            print("Botão 'Exportar dados' clicado. Aguardando 30s...")
            time.sleep(30)


        # -------------------------
        # Confirmar exportação
        # -------------------------
        exportar_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Exportar']"))
        )
        safe_click(exportar_button, driver)
        print("Exportação iniciada ✅")
        time.sleep(60)

        # -------------------------
        # Renomear arquivo baixado
        # -------------------------
        rename_downloaded_file()
        time.sleep(30)

    except Exception as e:
        print(f"⚠️ Erro na exportação: {e}")


def rename_downloaded_file():
    download_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"
    current_date = datetime.now().strftime("%Y%m")

    print("\U0001F4C2 Aguardando o download do arquivo...")

    # Tempo máximo de espera pelo download (60s)
    timeout = 60
    start_time = time.time()
    
    while True:
        files_in_folder = os.listdir(download_folder)
        excel_files = [f for f in files_in_folder if "data" in f.lower() and f.endswith(".xlsx")]
        
        if excel_files:
            file_name = excel_files[0]  # Pega o primeiro arquivo encontrado
            break
        
        if time.time() - start_time > timeout:
            print("\u26A0\uFE0F Tempo limite atingido. Nenhum arquivo encontrado para renomear.")
            return
        
        time.sleep(30)  # Espera 30 segundos antes de checar novamente

    base_name = f"AcompFreq_{current_date}"
    new_name = f"{base_name}.xlsx"
    old_file_path = os.path.join(download_folder, file_name)
    new_file_path = os.path.join(download_folder, new_name)

    counter = 1
    while os.path.exists(new_file_path):
        new_name = f"{base_name}_{counter}.xlsx"
        new_file_path = os.path.join(download_folder, new_name)
        counter += 1

    os.rename(old_file_path, new_file_path)
    print(f"\U0001F4C2 Arquivo salvo como {new_name}")

print("\U0001F4BB Abrindo o navegador...")
driver = webdriver.Chrome()
driver.get(url)
time.sleep(30)

print("\U0001F4DD Preenchendo o e-mail...")
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Aguardamos o título da página carregar
#WebDriverWait(driver, 30).until(EC.title_contains("Entrar") or EC.title_contains("Sign in"))
#print("✅ Página carregada.")



#email_input = wait.until(EC.presence_of_element_located((By.ID, "i0116")))
#email_input.send_keys("marketing.bi@visionone.com.br")
#email_input.send_keys(Keys.RETURN)

#iframes = driver.find_elements(By.TAG_NAME, "iframe")
#print(f"🔍 iframes encontrados: {len(iframes)}")

#for index, iframe in enumerate(iframes):
#    print(f"[{index}] src: {iframe.get_attribute('src')}")

wait = WebDriverWait(driver, 30)

# 1. Aguardamos o título da página carregar
wait.until(lambda d: "Entrar" in d.title or "Sign in" in d.title)
print("✅ Página carregada.")

# 2. Detecta iframes na página
iframes = driver.find_elements(By.TAG_NAME, "iframe")
print(f"🔍 iframes encontrados: {len(iframes)}")
for index, iframe in enumerate(iframes):
    print(f"[{index}] src: {iframe.get_attribute('src')}")

# 3. Se houver iframes, entra no primeiro iframe
if len(iframes) > 0:
    driver.switch_to.frame(iframes[0])
    print("🧩 Entrou no iframe 0")

try:
    # Tenta localizar pelo ID "email"
    email_input = wait.until(EC.visibility_of_element_located((By.ID, "email")))
    print("Campo de e-mail localizado pelo ID.")
except TimeoutException:
    print("ID 'email' não encontrado, tentando pelo placeholder...")
    # Tenta localizar pelo placeholder "Enter email"
    email_input = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[placeholder='Enter email']")))
    print("Campo de e-mail localizado pelo placeholder.")

# Preenche o e-mail e envia
email_input.send_keys("marketing.bi@visionone.com.br")
email_input.send_keys(Keys.RETURN)

print("📧 E-mail enviado.")

# 2. Aguarda o campo de senha ficar visível (pode demorar!)
password_input = wait.until(EC.visibility_of_element_located((By.ID, "i0118")))
password_input.send_keys("MkT_B1i@")
password_input.send_keys(Keys.RETURN)

print("🔐 Senha enviada. Aguardando botão de confirmação...")

time.sleep(5)

# 3. Aguarda o botão 'Sim' (confirmar permanência logado)
submit_button = wait.until(EC.element_to_be_clickable((By.ID, "idSIButton9")))
submit_button.click()

print("✅ Login concluído.")


print("🔄 Aguardando botão de confirmação aparecer...")

# Aguarda até o botão "Sim" (idSIButton9) estar clicável após o envio da senha
#submit_button = wait.until(EC.element_to_be_clickable((By.ID, "idSIButton9")))

# Clica no botão "Sim" para continuar
#submit_button.click()

print("✅ Login enviado com sucesso.")


time.sleep(60)
#submit_button = driver.find_element(By.ID, "idSIButton9")
#submit_button.click()
time.sleep(60)

print("\U0001F510 Login concluído com sucesso!")

reset_filters(driver)
print("\U0001F4C2 Iniciando exportação...")
export_data(driver)

print("\u2705 Processo concluído.")
driver.quit()

💻 Abrindo o navegador...
📝 Preenchendo o e-mail...
✅ Página carregada.
🔍 iframes encontrados: 0
Campo de e-mail localizado pelo ID.
📧 E-mail enviado.
🔐 Senha enviada. Aguardando botão de confirmação...
✅ Login concluído.
🔄 Aguardando botão de confirmação aparecer...
✅ Login enviado com sucesso.
🔐 Login concluído com sucesso!
🔹 Resetando filtros...
🔹 Filtros resetados.
⚠️ Erro ao resetar filtros: Message: 

📂 Iniciando exportação...
Clicando na coluna Última Admissão (1º clique) e esperando 60s...
Clicando na coluna Última Admissão (2º clique) e esperando 60s...
Clique adicional realizado. Aguardando 30s...
Menu 'Mais opções' aberto. Aguardando 1s para renderização...
Botão 'Exportar dados' clicado. Aguardando 30s...
Exportação iniciada ✅
📂 Aguardando o download do arquivo...
📂 Arquivo salvo como AcompFreq_202509.xlsx
✅ Processo concluído.


# MOVER ARQUIVOS DO DOWNLOADS PARA PASTA DE ACOMP FREQ SOLUCX

# SUBSTITUINDO TODOS OS DIAS PARA NÃO ACUMULAR

In [5]:
# Obter a data atual para o nome do arquivo
current_date = datetime.now().strftime("%Y%m")

# Diretório de downloads
# download_folder = r"C:\Users\HenriquedaLuzPacheco\Downloads"  # colocar caminho gaby downloads
download_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"  # Diretório de downloads

# Caminho da pasta de destino (você pode definir conforme necessário)
# destination_folder = r"" colocar caminho gaby pasta acomp freq solucx
destination_folder = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq"  # Pasta onde os arquivos serão movidos

# Procurar os arquivos baixados com o formato "AcompFreq_{data}"
files_in_folder = os.listdir(download_folder)

for file_name in files_in_folder:
    # Verifica se o nome do arquivo está no formato esperado
    if file_name.lower().startswith("acompfreq_") and file_name.endswith(f"_{current_date}.xlsx"):
        # Caminho completo do arquivo original e o destino final
        old_file_path = os.path.join(download_folder, file_name)
        new_file_path = os.path.join(destination_folder, file_name)

        # Criar a pasta de destino caso não exista
        os.makedirs(destination_folder, exist_ok=True)

        # Mover o arquivo para a pasta apropriada
        shutil.move(old_file_path, new_file_path)
        print(f"📂 Arquivo {file_name} movido para {destination_folder}")

📂 Arquivo AcompFreq_202509.xlsx movido para C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq


# CONSOLIDAR OS ARQUIVOS EXCEL EM UM ÚNICO, TODOS OS DIAS, PARA EVITAR LENTIDÃO NA LEITURA

# E EXCLUINDO OS OUTROS ARQUIVOS EXCEL

In [6]:
time.sleep(60)

In [7]:
# Pasta onde os arquivos estão localizados
base_folder = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq"

from datetime import timedelta

# Lista das colunas que serão utilizadas
colunas_desejadas = ['Cód. Paciente', 'Celular', 'E-mail', 'Primeira Admissão', 'Última Admissão', 'Unidade', 'Setor', 'Especialidade']

def load_and_save_acompfreq_data():
    print(f"Processando arquivos da pasta: {base_folder}")
    time.sleep(10)
    
    dfs = []
    
    try:
        # Carregar todos os arquivos Excel da pasta
        for file in os.listdir(base_folder):
            if file.endswith(".xlsx"):
                file_path = os.path.join(base_folder, file)
                print(f"Carregando arquivo: {file_path}")
                
                try:
                    df = pd.read_excel(file_path, usecols=colunas_desejadas)  # Apenas colunas desejadas
                    dfs.append(df)
                except Exception as e:
                    print(f"Erro ao ler {file}: {e}")
                
                time.sleep(10)
        
        if dfs:
            # Concatenar todos os dados em um único DataFrame
            df_acompfreq = pd.concat(dfs, ignore_index=True).drop_duplicates()
            print(f"Dados carregados e consolidados.")
            time.sleep(30)

            # Ensure datetime format
            df_acompfreq['Última Admissão'] = pd.to_datetime(
                df_acompfreq['Última Admissão'], 
                errors='coerce'
            )
            
            # Yesterday's date (no time portion)
            yesterday = pd.Timestamp(datetime.today().date() - timedelta(days=1))

            # Replace future dates with yesterday
            df_acompfreq.loc[df_acompfreq['Última Admissão'] > yesterday, 'Última Admissão'] = yesterday
            
            # Gerar um nome único para o arquivo consolidado
            base_filename = f"AF_Consolidado_{get_today_number()}"
            
            # Salvar os dados consolidados
            save_split_xlsx(df_acompfreq, base_folder, base_filename)
            time.sleep(10)
            
            # Excluir os arquivos antigos, movendo para a lixeira
            delete_old_files(base_folder, base_filename)
            time.sleep(10)
        else:
            print(f"Sem dados para processar.")
    
    except Exception as e:
        print(f"Erro geral no processamento: {e}")

def save_split_xlsx(df, output_folder, base_filename, max_rows=1048576):
    try:
        total_rows = df.shape[0]
        num_files = math.ceil(total_rows / max_rows)
        
        print(f"Total de {total_rows} linhas. Dividindo em {num_files} arquivos.")
        time.sleep(10)
        
        for i in range(num_files):
            start_row = i * max_rows
            end_row = min((i + 1) * max_rows, total_rows)
            output_file = os.path.join(output_folder, f"{base_filename}_Pt{i+1}.xlsx")
            df.iloc[start_row:end_row].to_excel(output_file, index=False)
            print(f"Arquivo salvo: {output_file} ({end_row - start_row} linhas)")
            time.sleep(10)
    
    except Exception as e:
        print(f"Erro ao salvar arquivos: {e}")

def delete_old_files(folder, current_filename):
    try:
        # Excluir todos os arquivos .xlsx, exceto o consolidado do dia atual
        for file in os.listdir(folder):
            if file.endswith(".xlsx") and current_filename not in file:
                file_path = os.path.join(folder, file)
                print(f"Excluindo arquivo: {file_path}")
                send2trash(file_path)  # Move para a lixeira
                time.sleep(10)
    
    except Exception as e:
        print(f"Erro ao excluir arquivos antigos: {e}")

def get_today_number():
    # Gerar número único com base na data para evitar conflitos
    today = datetime.today().strftime('%Y%m%d')
    return today

# Processar e salvar dados
load_and_save_acompfreq_data()

Processando arquivos da pasta: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq
Carregando arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq\AcompFreq_202509.xlsx


C:\Users\MarceloFridmanMarizd\anaconda3_new\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Carregando arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq\AF_Consolidado_20250908_Pt1.xlsx
Dados carregados e consolidados.
Total de 191599 linhas. Dividindo em 1 arquivos.
Arquivo salvo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq\AF_Consolidado_20250909_Pt1.xlsx (191599 linhas)
Excluindo arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq\AcompFreq_202509.xlsx
Excluindo arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - ESTRATÉGIA\SOLUCX\Rodando\Qvis\AcompFreq\AF_Consolidado_20250908_Pt1.xlsx


In [8]:
#!/usr/bin/env python
# coding: utf-8

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Função para enviar e-mails via Gmail
def enviar_email(remetente, senha, destinatarios, assunto, corpo):
    try:
        message = MIMEMultipart()
        message['Subject'] = assunto
        message['From'] = remetente
        message['To'] = ', '.join(destinatarios)
        
        # Adicionando o corpo do e-mail
        message.add_header('Content-Type', 'text/html')
        message.set_payload(corpo)

        # Conexão com o servidor SMTP do Gmail
        context = ssl.create_default_context()
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls(context=context)  # Habilita TLS (criptografia)
            server.login(remetente, senha)    # Login com a senha de app
            server.sendmail(
                remetente, destinatarios,
                message.as_string().encode('utf_8')
            )
        print(f"E-mail enviado com sucesso de {remetente}.")
        return True
    except Exception as e:
        print(f"Falha ao enviar e-mail com {remetente}. Erro: {e}")
        return False

# Definir destinatários
email_to = ['marcelo.carvalho@visionone.com.br', 'gabrielly.vetore@visionone.com.br']

# Corpo do e-mail (mensagem fixa)
corpo = r'''
    <p>Prezado(a),</p>
    <p>SUCESSO AO EXTRAIR ACOMP FREQ QVIS</p>
    <p>Esta é uma mensagem enviada automaticamente. Por favor, não a responda.</p>
'''

# Assunto do e-mail
assunto = 'SUCESSO ACOMP FREQ QVIS'

# Remetente e senha de aplicativo para Gmail
remetente_gmail = 'mcc.visionone@gmail.com'
senha_gmail = 'bqwk zcgm vgeh bcgp'  # Substitua com sua senha de aplicativo gerada

# Tentar enviar com Gmail
sucesso = enviar_email(remetente_gmail, senha_gmail, email_to, assunto, corpo)

if sucesso:
    print("E-mail enviado com sucesso!")
else:
    print("Falha ao enviar e-mail.")

E-mail enviado com sucesso de mcc.visionone@gmail.com.
E-mail enviado com sucesso!


In [9]:
fim_tempo = datetime.now()
tempo = fim_tempo - inicio_tempo

# Converter para minutos e segundos
minutos, segundos = divmod(tempo.total_seconds(), 60)

print(f"Tempo de execução: {int(minutos)} minutos e {segundos:.2f} segundos")

Tempo de execução: 14 minutos e 38.58 segundos
